In [1]:
%pwd

'/ocean/projects/asc170022p/mtragoza/MRE-PINN'

In [11]:
%load_ext autoreload
%autoreload 1
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys, os
os.environ['DDEBACKEND'] = 'pytorch'
import numpy as np
import matplotlib.pyplot as plt

%aimport mre_pinn

In [14]:
%autoreload
wave_file = 'data/wave_sim/steady_state_wave.npy'
wave_image = mre_pinn.ImagePointSet(wave_file)

/ocean/projects/asc170022p/mtragoza/.conda/envs/MRE-PINN/lib/python3.8/site-packages/deepxde/backend/pytorch/tensor.py:64: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853042/work/aten/src/ATen/native/Copy.cpp:239.)
  return torch.as_tensor(data, dtype=dtype)


In [18]:
wave_image

In [54]:
import torch
import deepxde

dtype = deepxde.config.real(torch)


def as_complex(t):
    real, imag = t[:,::2], t[:,1::2]
    return torch.complex(real, imag)


class ComplexFFN(torch.nn.ModuleList):
    
    def __init__(self, n_input, n_layers, n_hidden, n_output, activ_fn):
        
        modules = []
        for i in range(n_layers):
            is_first_layer = (i == 0)
            is_last_layer = (i == n_layers - 1)
            linear = torch.nn.Linear(
                n_input if is_first_layer else n_hidden,
                n_output*2 if is_last_layer else n_hidden
            )
            modules.append(linear)
    
        self.activ_fn = activ_fn
        super().__init__(modules)
        
    def forward(self, input):
        
        for i, module in enumerate(self):
            if i < len(self) - 1:
                input = self.activ_fn(module(input))
            else:
                output = as_complex(module(input))
        return output


class Parallel(torch.nn.ModuleList):
    
    def forward(self, input):
        outputs = [module(input) for module in self]
        return torch.cat(outputs, dim=1)


u_net  = ComplexFFN(n_input=2, n_layers=1, n_hidden=3, activ_fn=torch.sin, n_output=2)
mu_net = ComplexFFN(n_input=2, n_layers=1, n_hidden=3, activ_fn=torch.sin, n_output=1)

Parallel([u_net, mu_net]) #(torch.eye(2))

Parallel(
  (0): ComplexFFN(
    (0): Linear(in_features=2, out_features=4, bias=True)
  )
  (1): ComplexFFN(
    (0): Linear(in_features=2, out_features=2, bias=True)
  )
)

In [49]:
t = np.arange(8)
t[0::2], t[1::2]

(array([0, 2, 4, 6]), array([1, 3, 5, 7]))